In [1]:
import sys

sys.path.insert(0, 'compiled_protobufs')

In [ ]:
# import IPython
# import sys
# from pygaggle.rerank.transformer import MonoT5
# !{sys.executable} -m pip install -r pygaggle/requirements.txt
# IPython.Application.instance().kernel.do_shutdown(True)  

In [2]:
""" Generate Taskgraphs from datasets """

from models_datasets.recipe_1m_model import AbstractModelDataset
from models_datasets.recipe_1m_model import Recipe1MModel
from models_datasets.wikihow_model import WikihowModel

dataset_models = {
    "COOKING" : Recipe1MModel,
    "DIY": WikihowModel,
}

for model_name, model in dataset_models.items():
    dataset_model: AbstractModelDataset = model()
    print(f"Dataset Model: {model_name}")
    dataset_model.convert_to_taskgraphs()

Dataset Model: COOKING
Taskgraphs already converted. Call overwrite=True in convert_to_taskgraphs() to overwrite the preprocessed taskgraphs.
Dataset Model: DIY
Taskgraphs already converted. Call overwrite=True in convert_to_taskgraphs() to overwrite the preprocessed taskgraphs.


In [5]:
import pandas as pd

""" Preprocess and load queries """

# from query_pipeline import query_pipeline
# query_pipeline.preprocess_queries()

queries = {
    "COOKING" : pd.read_csv('queries/cooking.csv'),
    "DIY": pd.read_csv('queries/diy.csv'),
}

queries["COOKING"].head()

,id,raw query,domain,knowledge,complex,many points of view,different key entities,seasonal,regional,target query,short query
0,query-0,how to spatchcock a turkey,cooking,n,y,n,n,y,usa,spatchcock turkey,spatchcock a turkey
1,query-1,I want an easy to make dessert for christmas,cooking,n,y,y,n,y,-,easi dessert christma,easy to make dessert for christmas
2,query-2,any recommendations for a gluten free appetizer,cooking,y,n,y,y,n,-,gluten free appet,gluten free appetizer
3,query-3,how to make a green goddess salad,cooking,y,y,n,y,n,usa,green goddess salad,green goddess salad
4,query-4,how to make a classic english trifle,cooking,y,n,n,y,y,uk,classic english trifl,classic english trifle


In [3]:
# from models_indexes.bm25_model import BM25Model
# from models_indexes.ance_model import AnceModel
# from models_indexes.colbert_model import ColbertModel
from models_indexes.marqo_model import MarqoModel
# from models_indexes.abstract_model import AbstractModel

models = {}

for domain in  ["COOKING", "DIY"]:
# for domain in  ["COOKING"]:
    print(domain)
    models[domain] = {
        # "bm25" : BM25Model(domain = domain),
        # "bm25+rm3" : BM25Model(domain = domain, rm3=True),
        # "bm25+t5" : BM25Model(domain = domain, t5=True),
        # "bm25+rm3+t5" : BM25Model(domain = domain, rm3=True, t5=True),
        # "ance+t5": AnceModel(domain = domain, t5=True),
        # "colbert+t5": ColbertModel(domain = domain, t5=True),
        "marqo": MarqoModel(domain = domain, t5 = True),
        # "marqo": MarqoModel(domain = domain),
    }

2023-02-06 22:11:31.015531: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-06 22:11:31.047654: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-06 22:11:31.048439: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


COOKING
DIY


In [ ]:
# build indexes
print("Build indexes")
for domain, index_models in models.items():
    for model_name, model in index_models.items():
        model.build_index()

In [6]:
# create run files
print("Creating run files ")
for domain, index_models in models.items():
    qs = queries[domain]
    print(f"DOMAIN {domain}")
    for model_name, model in index_models.items():
        model.convert_search_results_to_run(qs)

Creating run files 
DOMAIN COOKING
query 0/100


2023-02-06 22:12:08,249 logger:'marqo' INFO search (tensor): took 0.271s to send query and received 50 results from Marqo (roundtrip). Marqo itself took 0.265s to execute the search.
2023-02-06 22:12:08 [INFO] index: search (tensor): took 0.271s to send query and received 50 results from Marqo (roundtrip). Marqo itself took 0.265s to execute the search.


scores [0.8912988, 0.8850584, 0.8751605, 0.8750533, 0.8711824, 0.8707428, 0.87001705, 0.8687583, 0.8684166, 0.86841655, 0.8662876, 0.865132, 0.8635899, 0.8626672, 0.86249316, 0.86164916, 0.8616049, 0.8613444, 0.86134434, 0.86119306, 0.8600724, 0.85976213, 0.85912144, 0.8590493, 0.8586949, 0.8585799, 0.85847366, 0.85724527, 0.85717857, 0.8568218, 0.85671866, 0.8563638, 0.85603863, 0.85573846, 0.8557336, 0.85569394, 0.85569227, 0.8556129, 0.85547125, 0.85496867, 0.8545394, 0.8544447, 0.8544198, 0.8542986, 0.8542187, 0.8536625, 0.8534535, 0.85341865, 0.8530189, 0.85295415]
ids ['cooking+recipe1m+994505df2dd754dc64bea1efeddd0cf4', 'cooking+recipe1m+b28a06daba651ea98a5425ec2dd94ba0', 'cooking+recipe1m+3d6783e694a2d1cc7ecc005f8aedf2b6', 'cooking+recipe1m+13febb82743ad0713b4bfe82c9f91453', 'cooking+recipe1m+104fe308f78ba2bd4512b9bcf15b86cc', 'cooking+recipe1m+e9263003ea034f10372a27ffd0d6c6bd', 'cooking+recipe1m+4171f386c3a6574ef242e97d5ad590ab', 'cooking+recipe1m+1f8b10db4893b0e87c801a036c8f3

2023-02-06 22:12:34,438 logger:'marqo' INFO search (tensor): took 14.432s to send query and received 50 results from Marqo (roundtrip). Marqo itself took 14.426s to execute the search.
2023-02-06 22:12:34 [INFO] index: search (tensor): took 14.432s to send query and received 50 results from Marqo (roundtrip). Marqo itself took 14.426s to execute the search.


scores [0.84303904, 0.8396511, 0.8385334, 0.8345282, 0.8311053, 0.83029294, 0.83018774, 0.8291454, 0.825866, 0.82484734, 0.82478714, 0.8247452, 0.82392627, 0.8237423, 0.8217777, 0.82146657, 0.8208376, 0.82064176, 0.8203597, 0.8200425, 0.8190958, 0.81887925, 0.8177892, 0.81742793, 0.8174275, 0.817427, 0.8173821, 0.81710577, 0.81687164, 0.81622404, 0.8152808, 0.81521094, 0.8150927, 0.8149381, 0.8147192, 0.81442034, 0.8142215, 0.8132379, 0.81295824, 0.811822, 0.8116487, 0.81144977, 0.81094706, 0.8106921, 0.81060207, 0.81007636, 0.8098476, 0.80945075, 0.80913913, 0.8090552]
ids ['cooking+recipe1m+3efbd1fa14a7f98f30d47f4b20b664ce', 'cooking+recipe1m+a1d518ac4951a10f198323ca07be1328', 'cooking+recipe1m+e6414e088ad92352614f7cb25bbfe29e', 'cooking+recipe1m+c495b85fdfcf330525932d44b558f75f', 'cooking+recipe1m+1c6c6e0020b330fd69af34c87ae3e187', 'cooking+recipe1m+3a38f316324e85bf691bc988894cc079', 'cooking+recipe1m+ceb7a068929c0e56252a7f80457176f5', 'cooking+recipe1m+19a214517eec2c422c10aedc5df29

...

scores [0.8714187, 0.8586315, 0.85844636, 0.85750616, 0.85692835, 0.852703, 0.8523639, 0.8519105, 0.84528315, 0.84149086, 0.84092635, 0.8408804, 0.8347074, 0.83401334, 0.8334526, 0.8334476, 0.8321829, 0.831405, 0.83051264, 0.8284446, 0.8272202, 0.8268336, 0.82609177, 0.8258041, 0.8257133, 0.8251631, 0.8245642, 0.82325506, 0.8211401, 0.8211056, 0.82021314, 0.81972426, 0.8194282, 0.81939983, 0.81827605, 0.81759644, 0.81581247, 0.8157402, 0.81549263, 0.81520915, 0.81364787, 0.8135672, 0.8133014, 0.8130833, 0.8128575, 0.8115318, 0.8114434, 0.8109853, 0.8089739, 0.8087595]
ids ['diy+wikihow-offline+395f2b556bd38aca9768f5cac270b5a2', 'diy+wikihow-offline+beb8194ee01f8f4e32426c339815bd3c', 'diy+wikihow-offline+d92530ad321f360b7727efb8f0e1adfb', 'diy+wikihow-offline+c22de0cf11435050faefc02e799e4449', 'diy+wikihow-offline+435a48257af4c81354ff2d0831656944', 'diy+wikihow-offline+dfe23f3881b3193dd11d1fa221ee886d', 'diy+wikihow-offline+d1dbde065da0e0fe6518ad82ee3afd6a', 'diy+wikihow-offline+31b4e10

2023-02-06 22:18:52,661 logger:'marqo' INFO search (tensor): took 0.205s to send query and received 50 results from Marqo (roundtrip). Marqo itself took 0.199s to execute the search.
2023-02-06 22:18:52 [INFO] index: search (tensor): took 0.205s to send query and received 50 results from Marqo (roundtrip). Marqo itself took 0.199s to execute the search.


scores [0.88087326, 0.87673795, 0.8623696, 0.8597511, 0.8561635, 0.8528098, 0.8504735, 0.84865355, 0.8462217, 0.84428555, 0.8437212, 0.8432985, 0.8409343, 0.8406987, 0.83885676, 0.8372206, 0.83677983, 0.83673024, 0.83570033, 0.83226544, 0.83168566, 0.8305518, 0.83051157, 0.82890123, 0.8284154, 0.826118, 0.82541984, 0.82457304, 0.8234165, 0.8230685, 0.8230512, 0.82258236, 0.8224173, 0.82216346, 0.8209717, 0.82021666, 0.82006407, 0.8198818, 0.8198357, 0.8195065, 0.8174579, 0.81731296, 0.8153087, 0.81444293, 0.81437504, 0.8138427, 0.8137412, 0.8129618, 0.81160975, 0.8109057]
ids ['diy+wikihow-offline+166cbe851b68c6457462de62f06d9c70', 'diy+wikihow-offline+984a3b8deda0e584142436b60857927d', 'diy+wikihow-offline+39dd674a1b5754b10bbcccee7aa988a9', 'diy+wikihow-offline+3b6729f35dbe95589f55f0ecd518e641', 'diy+wikihow-offline+0a173dfe2ff4530b7b8a20c0f7a505a6', 'diy+wikihow-offline+a523df9bd1326a03741fd7fb85d83d81', 'diy+wikihow-offline+6ffa302845cc06c7ceed5addcf301d9f', 'diy+wikihow-offline+dd7

2023-02-06 22:18:54,196 logger:'marqo' INFO search (tensor): took 0.213s to send query and received 50 results from Marqo (roundtrip). Marqo itself took 0.207s to execute the search.
2023-02-06 22:18:54 [INFO] index: search (tensor): took 0.213s to send query and received 50 results from Marqo (roundtrip). Marqo itself took 0.207s to execute the search.


scores [0.928753, 0.8882657, 0.882401, 0.87731814, 0.87614316, 0.8738941, 0.87283534, 0.8564018, 0.84629124, 0.84589183, 0.8453802, 0.8419286, 0.8392652, 0.83841765, 0.83429706, 0.82583165, 0.82532835, 0.8246605, 0.8214724, 0.8203563, 0.8185382, 0.817001, 0.8145138, 0.8124794, 0.81206137, 0.8107089, 0.81044596, 0.809778, 0.80873346, 0.8072103, 0.80633056, 0.8062702, 0.80542827, 0.80425155, 0.8039385, 0.803289, 0.8029665, 0.7996516, 0.79931164, 0.79915744, 0.79827124, 0.7950741, 0.79363626, 0.79232776, 0.79174674, 0.7905723, 0.7898866, 0.78936386, 0.78533804, 0.7846446]
ids ['diy+wikihow-offline+ee3520f7d7889a3d3e81ab575aced8ce', 'diy+wikihow-offline+8bcfef8ba08280fe6a0d76e6f07f2944', 'diy+wikihow-offline+19f0cfb05f765610a325c0dc6823c4f0', 'diy+wikihow-offline+72793061ef42b2f74f1f497c0e74908a', 'diy+wikihow-offline+2d5d89d970e75e99026ff2e371c661f0', 'diy+wikihow-offline+a525b9c68c28642f45a5e23a642dd844', 'diy+wikihow-offline+c47d465556e57e95d7d6370d80c2e981', 'diy+wikihow-offline+d10f46